In [ ]:
import networkx as nx
from Bio.PDB import *
from eden.util import read
from GArDen.interfaces import convert, transform

def extract_residues(structure):
    res_seq = []
    for model in structure:
        for chain in model:
            for res in chain:
                atoms = [atom for atom in res if atom.get_name()=='CA']
                if atoms:
                    ca_atom = atoms[0]
                    res_seq.append((res.get_resname(), ca_atom))
    return res_seq

def extract_ligand_atoms(structure, ligand_code):
    res_seq = []
    for model in structure:
        for chain in model:
            for res in chain:
                for atom in res:
                    if res.get_resname() == ligand_code:
                        #print atom.get_name()+':'+str(atom.get_vector()[0])+':'+str(atom.get_vector()[1])+':'+str(atom.get_vector()[2])
                        res_seq.append(atom)
    if (res_seq == []):
        print "ERROR: ligand code not found"
    return res_seq

def ac_encoding(code, scheme='code_20'):
    ac_dict = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
               'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
               'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
               'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}
    # codes from L. R. Murphy, A. Wallqvist, and R. M. Levy, “Simplified amino acid alphabets for protein fold recognition and implications for folding,” Protein Eng., vol. 13, no. 3, pp. 149–152, Mar. 2000.
    code_3 = ['LASGVTIPMC', 'EKRDNQH', 'FYW']
    code_5 = ['LVIMC', 'ASGTP', 'FYW', 'EDNQ', 'KRH']
    code_6 = ['LVIM', 'ASGT', 'PHC', 'FYW', 'EDNQ', 'KR']
    code_12 = ['LVIM', 'C', 'A', 'G', 'ST', 'P', 'FY', 'W', 'EQ', 'DN', 'KR', 'H']
    
    if ac_dict.get(code,False) is False:
        return code
    ac_1letter_code = ac_dict[code]
    if scheme=='code_20':
        return ac_1letter_code
    elif scheme=='code_3':
        for n,codes in  enumerate(code_3):
            if ac_1letter_code in codes: return n
    elif scheme=='code_5':
        for n,codes in  enumerate(code_5):
            if ac_1letter_code in codes: return n
    elif scheme=='code_6':
        for n,codes in  enumerate(code_6):
            if ac_1letter_code in codes: return n
    elif scheme=='code_12':
        for n,codes in  enumerate(code_12):
            if ac_1letter_code in codes: return n
    else:
        raise Exception('Unknown scheme: %s'%scheme)

def trim_graph_to_knn_nesting(g, knn=4):
    for u in g.nodes():
        neighs = g.neighbors(u)
        lens = [g.edge[u][v]['len'] for v in neighs]
        if len(lens) > knn:
            th = sorted(lens)[knn]
            for v in neighs:
                if g.edge[u][v]['len'] > th:
                    g.edge[u][v]['nesting']=True

def trim_graph_to_dist_nesting(g, distance_threshold2=4):
    for u in g.nodes():
        neighs = g.neighbors(u)
        for v in neighs:
            if g.edge[u][v]['len'] > distance_threshold2:
                g.edge[u][v]['nesting']=True


def make_graph(structure, distance_threshold=5, distance_threshold2=4, scheme='code_20'):
    res_seq = extract_residues(structure)

    g = nx.Graph()
    for i, (ac,atom) in enumerate(res_seq):
        g.add_node(i, label=ac_encoding(ac, scheme=scheme), name=ac, atom=atom)

    for u in g.nodes():
        for v in g.nodes():
            dist = g.node[u]['atom'] - g.node[v]['atom']
            if 0 < dist < distance_threshold:
                g.add_edge(u,v, label='-', len=dist, weigth=1/float(dist))
    trim_graph_to_dist_nesting(g, distance_threshold2=distance_threshold2)
    return g

# TODO: retrieve PDB, retrieve protein CA and ligand atoms
def make_interaction_graph(structure, ligand_code, distance_threshold=5, distance_threshold2=4, scheme='code_20'):
    ligand_atom_list = extract_ligand_atoms(structure, ligand_code)
    residue_atom_list = extract_residues(structure) # does it only extract the amino acids of the PDB or also the potential ligands?
    
    g = nx.Graph()
    for i, (ac,atom) in enumerate(residue_atom_list):
        g.add_node(i, label=ac_encoding(ac, scheme=scheme), name=ac, atom=atom)
    
    for a, atom in enumerate(ligand_atom_list): 
        #TODO: which information to store in ligand atoms?
        g.add_node(a, label=atom.element, name=ligand_code, atom=atom) # TODO: encoding these nodes

    for u in g.nodes():
        for v in g.nodes():
            dist = g.node[u]['atom'] - g.node[v]['atom']
            if 0 < dist < distance_threshold:
                g.add_edge(u,v, label='-', len=dist, weigth=1/float(dist))
                # TODO: do we also use nested edges to represent the links between ligand atoms and protein atoms?
                
    return g

graphs = []
# read in PDB files and retrieve the ligand codes
for line in read('INDEX_refined_data.2015_temp'):
    if not(line.startswith('#')): # skip info lines
        parts = line.split()
        pdb_id = parts[0]
        ligand = parts[7].strip('()')
        if '-' not in ligand: # TODO: figure out what to do with peptides and if there are multiple ligands
            if '&' not in ligand:
                #print 'PDB:'+pdb_id+', Ligand:'+ligand
                print 'Retrieving '+pdb_id+'...'
                fname = PDBList().retrieve_pdb_file(pdb_id,pdir='pdbs') # pdir is the directory to store the PDBs
                structure = PDBParser().get_structure('X', fname) # why X?
                g = make_interaction_graph(structure,ligand)
                graphs.append(g)
                
# toy example


#fname = PDBList().retrieve_pdb_file('2r58')
#structure = PDBParser().get_structure('X', fname)
#g0 = make_interaction_graph(structure,'MLY')

#print extract_residues(structure)[0]
#extract_ligand_atoms(structure, 'MLY')
#th=3
#g = make_graph(structure, distance_threshold=10, distance_threshold2=th, scheme='code_6')
#draw_graph(g, **display_params)


Retrieving 2r58...
Structure exists: 'pdbs/pdb2r58.ent' 
Retrieving 3c2f...
Structure exists: 'pdbs/pdb3c2f.ent' 
Retrieving 3g2y...
Structure exists: 'pdbs/pdb3g2y.ent' 


/Users/leander/anaconda/envs/eden_env/lib/python2.7/site-packages/Bio/PDB/PDBParser.py:196: PDBConstructionWarning: Negative occupancy in one or more atoms
  warnings.warn("Negative occupancy in one or more atoms", PDBConstructionWarning)
/Users/leander/anaconda/envs/eden_env/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 8589.
  PDBConstructionWarning)
/Users/leander/anaconda/envs/eden_env/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 8617.
  PDBConstructionWarning)
/Users/leander/anaconda/envs/eden_env/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 8653.
  PDBConstructionWarning)
/Users/leander/anaconda/envs/eden_env/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 9281.
  P

Retrieving 3pce...
Structure exists: 'pdbs/pdb3pce.ent' 


/Users/leander/anaconda/envs/eden_env/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructionWarning: WARNING: Chain M is discontinuous at line 21914.
  PDBConstructionWarning)
/Users/leander/anaconda/envs/eden_env/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructionWarning: WARNING: Chain N is discontinuous at line 21930.
  PDBConstructionWarning)
/Users/leander/anaconda/envs/eden_env/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructionWarning: WARNING: Chain O is discontinuous at line 21946.
  PDBConstructionWarning)
/Users/leander/anaconda/envs/eden_env/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructionWarning: WARNING: Chain P is discontinuous at line 21962.
  PDBConstructionWarning)
/Users/leander/anaconda/envs/eden_env/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructionWarning: WARNING: Chain Q is discontinuous at line 21978.
  PDBConstructionWarning)
/Users/leander/

In [ ]:
from eden.util.display import draw_graph_set
from GArDen.transform.node import ColorNode
parameters_priors = dict(output_attribute='level', labels=[['C','O','N','S','H','Cl','.C','.O','.N','.S','.H','.Cl']])
draw_graphs = transform(graphs, program=ColorNode(), parameters_priors=parameters_priors)
draw_graph_set(draw_graphs, n_graphs_per_line=2, size=20, title_key='id', prog='sfdp', node_border=1, node_size=200, colormap='Set3',
               edge_color='_label_',vertex_alpha=1, edge_alpha=.4, vertex_label='label', #secondary_vertex_label='type', 
               vertex_color='level', ignore_for_layout='nesting')

/Users/leander/anaconda/envs/eden_env/lib/python2.7/site-packages/pygraphviz/agraph.py:1338: RuntimeWarning: Error: remove_overlap: Graphviz not built with triangulation library

  warnings.warn(b"".join(errors), RuntimeWarning)
